In [1]:
import parse
import watermark 
import portion as por

In [2]:
example = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""

In [3]:
def map_seed(seed, s):
    instruct = parse.parse("{key1}-to-{key2} map:{maps}", s)
    loc_in = seed[instruct["key1"]]
    maps = instruct["maps"].strip().splitlines()
    updated = False

    #print(instruct["key1"])
    #print("loc_in: " + str(loc_in))

    for m in maps:
        out_start, in_start, map_range = [int(i) for i in m.split()]
        #Check if loc_in is in the in_range
        if loc_in in range(in_start, in_start + map_range):
            seed[instruct["key2"]] = loc_in - in_start + out_start
            updated = True

    if not updated:
        seed[instruct["key2"]] = seed[instruct["key1"]]

    return seed

In [5]:
def seed_gen_ind(seed_sp):
    for i in parse.parse("seeds: {}", seed_sp)[0].split():
        yield i

In [6]:
def seed_gen_range(seed_sp):
    seed_spec = [int(i) for i in parse.parse("seeds: {}", seed_sp)[0].split()]
    for i in range(0, len(seed_spec), 2):
        for j in range(seed_spec[i], seed_spec[i] + seed_spec[i+1]):
            yield j

## Part 1

In [7]:
input = example.split("\n\n")

seed_list = [{"seed" : int(i)} for i in seed_gen_ind(input[0])]

for seed in seed_list:
    for map in input[1:]:
        seed = map_seed(seed, map)

print(min(seed["location"] for seed in seed_list))

35


In [8]:
with open("input.txt") as f:
    input = f.read().split("\n\n")

seed_list = [{"seed" : int(i)} for i in seed_gen_ind(input[0])]

for seed in seed_list:
    for map in input[1:]:
        seed = map_seed(seed, map)
        #print(map)

print(min(seed["location"] for seed in seed_list))

196167384


## Part 2

In [12]:
def seed_gen_interval(seed_sp):
    seed_spec = [int(i) for i in parse.parse("seeds: {}", seed_sp)[0].split()]
    tmp = por.empty()

    for start, range in zip(seed_spec[::2], seed_spec[1::2]):
        tmp = tmp.union(por.closedopen(start, start + range))
        
    return tmp

In [9]:
def map_seed_int(seed, s):
    instruct = parse.parse("{key1}-to-{key2} map:{maps}", s)
    in_terval = seed[instruct["key1"]]
    maps = instruct["maps"].strip().splitlines()

    total_mapped = por.empty()
   
    for map_spec in maps:
        # Get the specs
        out_start, in_start, m_range = [int(i) for i in map_spec.split()]
        
        # Compute the domain of the map and the bias
        bias = out_start - in_start
        dom_int = por.closedopen(in_start, in_start + m_range)
       
        # Figure out which of the interval is affected
        aff_int = in_terval.intersection(dom_int)
        # Add it to the total mapped interval
        total_mapped =  total_mapped | aff_int.apply(lambda x: (x.left, x.lower + bias, x.upper + bias, x.right))
        
        # Remove it from the interval yet to be affected
        in_terval = in_terval - aff_int

    seed[instruct["key2"]] = in_terval | total_mapped

    return seed

In [20]:
input = example.split("\n\n")

seed = {"seed" : seed_gen_interval(input[0])}

for map in input[1:]:
    seed = map_seed_int(seed, map)


seed["location"].lower

46

In [22]:
%%timeit
with open("input.txt") as f:
    input = f.read().split("\n\n")

seed = {"seed" : seed_gen_interval(input[0])}

for map in input[1:]:
    seed = map_seed_int(seed, map)


seed["location"].lower

31.8 ms ± 39.8 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
